#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020

# Spotlight: Deeper investigation of some neat aspects of pandas

### By: Tianyu Huang

### Due: April 9, 2020

## Introduction
In computer programming, pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series. It is free software released under the three-clause BSD license. The name is derived from the term "panel data", an econometrics term for data sets that include observations over multiple time periods for the same individuals.

Pandas is mainly used for machine learning in form of dataframes. Pandas allow importing data of various file formats such as csv, excel etc. Pandas allows various data manipulation operations such as groupby, join, merge, melt, concatenation as well as data cleaning features such as filling, replacing or imputing null values.

This article mainly introduces some advanced functions and usages that are commonly used, including groupby(), apply(), pivot_table(), time series operations and string operations.

## Import Package

In [4]:
import pandas as pd
pd.__version__

'0.25.1'

## 1: groupby()

* Regardless of whether the grouping key is an array, list, dictionary, series, function, as long as it is the same as the axis length of the variable to be grouped, it can be passed into groupby() for grouping.
* The default axis = 0 is grouped by row, and you can specify axis = 1 to group columns.

In [93]:
# form a dataframe
data = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                     'value':[0,5,10,5,10,15,10,15,20]})
print(data)
print(data.groupby('key').sum())

  key  value
0   A      0
1   B      5
2   C     10
3   A      5
4   B     10
5   C     15
6   A     10
7   B     15
8   C     20
     value
key       
A       15
B       30
C       45


The above is just a sum operation, of course, you can also use many statistical methods: size(), count(), max(), min(), mean(), etc.

The point to note is the difference between the size() method and the count() method. Size() directly counts the number of occurrences of the key, so there will be no value column in the result, and the size() method will count missing values, and count() will not.

In [94]:
import numpy as np

data2 = pd.DataFrame({'key':['A','B','C','A','B'],'value':[0,5,10,5,np.nan]})
print(data2)
print(data2.groupby('key').size())
print(data2.groupby('key').count())

  key  value
0   A    0.0
1   B    5.0
2   C   10.0
3   A    5.0
4   B    NaN
key
A    2
B    2
C    1
dtype: int64
     value
key       
A        2
B        1
C        1


In addition, you can take out the group alone to do some operations.

In [95]:
# First grouped, but no aggregation operation, the result is a groupby object
groups = data.groupby('key')
print(groups)

# Can follow the dictionary format to traverse
for index,group in groups:
    print('---------------')
    print(index,'\n',group)
    print('-------------')
    
    # mean()
    print(group.mean())

---------------
A 
   key  value
0   A      0
3   A      5
6   A     10
-------------
value    5.0
dtype: float64
---------------
B 
   key  value
1   B      5
4   B     10
7   B     15
-------------
value    10.0
dtype: float64
---------------
C 
   key  value
2   C     10
5   C     15
8   C     20
-------------
value    15.0
dtype: float64


### get_group()
Use this method to select the desired group, key value index

In [96]:
data3 = pd.DataFrame({'key':['A','B','A','B'],'value':[0,5,10,5]})
print(data3)
print(data3.groupby('key').get_group('A'))
print(data3.groupby('key').get_group('B'))

  key  value
0   A      0
1   B      5
2   A     10
3   B      5
  key  value
0   A      0
2   A     10
  key  value
1   B      5
3   B      5


### Multiple index
In many cases, we will encounter the problem of multiple indexes, how to use groupby better, you can formulate level parameters

level = 0 means only the first index is used, level = 1 means only the second index is used, and so on, and the index name can also be specified directly.

In [97]:
# Use pandas method to generate a series of secondary index structure
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays,names = ['first','second'])
s = pd.Series(np.random.randn(8),index = index)
print(s)
print('------------------')
print(s.groupby(level =0).sum())
print('------------------')
print(s.groupby(level =1).sum())
print('------------------')
print(s.groupby(level ='first').sum())

first  second
bar    one       0.139614
       two       0.526843
baz    one      -0.108668
       two       0.796672
foo    one      -2.213745
       two       1.569220
qux    one       0.631751
       two       0.489118
dtype: float64
------------------
first
bar    0.666457
baz    0.688004
foo   -0.644525
qux    1.120869
dtype: float64
------------------
second
one   -1.551048
two    3.381854
dtype: float64
------------------
first
bar    0.666457
baz    0.688004
foo   -0.644525
qux    1.120869
dtype: float64



## 2: Custom function
In pandas, you can use apply, map, agg to perform custom function operations.

In [100]:
# Define a function, the role is to increase the value by 1
data3 = pd.DataFrame({'key':['A','B','A','B'],'value':[0,5,10,5]})
print(data3)

def add_1(value):
    return value + 1

print("----------------")
print(data3['value'].apply(add_1))
print(data3['value'].map(add_1))
print(data3['value'].agg(add_1))

print('----------------')
print(data3.groupby('key').apply(add_1))
print(data3.groupby('key').agg({'value':np.mean}))

  key  value
0   A      0
1   B      5
2   A     10
3   B      5
----------------
0     1
1     6
2    11
3     6
Name: value, dtype: int64
0     1
1     6
2    11
3     6
Name: value, dtype: int64
0     1
1     6
2    11
3     6
Name: value, dtype: int64
----------------
   value
0      1
1      6
2     11
3      6
     value
key       
A        5
B        5


## 3: Pivot table
Pivot Table is an interactive table that can perform certain calculations, such as summing and counting. The calculations and data are related to the arrangement in the pivot table.

They are called pivot tables because they can dynamically change their layout to analyze data in different ways, and they can also rearrange row numbers, column labels, and page fields. Every time the layout is changed, the PivotTable will immediately recalculate the data according to the new layout. In addition, if the original data changes, you can update the PivotTable.
Pandas uses pivot_table() method to generate pivot table.

A simple example of summation:

In [102]:
# form a dataframe
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                          "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
print(df)
print('--------------------------')
# Take A and B as row indexes and C as column indexes to sum D
print(pd.pivot_table(df, values='D', index=['A', 'B'],
                  columns=['C'], aggfunc=np.sum))

     A    B      C  D  E
0  foo  one  small  1  2
1  foo  one  large  2  4
2  foo  one  large  2  5
3  foo  two  small  3  5
4  foo  two  small  3  6
5  bar  one  large  4  6
6  bar  one  small  5  8
7  bar  two  small  6  9
8  bar  two  large  7  9
--------------------------
C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0


Fill missing value as 0:

In [103]:
print(pd.pivot_table(df, values='D', index=['A', 'B'],
                     columns=['C'], aggfunc=np.sum, fill_value=0))

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6


Take multiple aggregations in multiple columns:

In [104]:
print(pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                   aggfunc={'D': np.mean,
                            'E': [min, max, np.mean]}))

                  D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0


## 4: Time series operations

### Time stamps and time series

In [105]:
ts = pd.Timestamp('2019-8-19')
print(ts)
print(pd.Series(pd.date_range(start='2019-8-19',periods = 10,freq = '12H')))

2019-08-19 00:00:00
0   2019-08-19 00:00:00
1   2019-08-19 12:00:00
2   2019-08-20 00:00:00
3   2019-08-20 12:00:00
4   2019-08-21 00:00:00
5   2019-08-21 12:00:00
6   2019-08-22 00:00:00
7   2019-08-22 12:00:00
8   2019-08-23 00:00:00
9   2019-08-23 12:00:00
dtype: datetime64[ns]


### Time index and indice

In [106]:
data = pd.read_csv('./flowdata.csv',index_col = 'Time',parse_dates = True)
# Time slice
print(data[('2012-01-01 09:00'):('2012-01-01 19:00')])
# year index
print(data['2013'])
# Index data from January to February 2012
print(data['2012-01':'2012-02'].head(10))

                      L06_347  LS06_347  LS06_348
Time                                             
2012-01-01 09:00:00  0.330750  0.293583  0.029750
2012-01-01 12:00:00  0.295000  0.285167  0.031750
2012-01-01 15:00:00  0.301417  0.287750  0.031417
2012-01-01 18:00:00  0.322083  0.304167  0.038083
                      L06_347  LS06_347  LS06_348
Time                                             
2013-01-01 00:00:00  1.688333  1.688333  0.207333
2013-01-01 03:00:00  2.693333  2.693333  0.201500
2013-01-01 06:00:00  2.220833  2.220833  0.166917
2013-01-01 09:00:00  2.055000  2.055000  0.175667
2013-01-01 12:00:00  1.710000  1.710000  0.129583
2013-01-01 15:00:00  1.420000  1.420000  0.096333
2013-01-01 18:00:00  1.178583  1.178583  0.083083
2013-01-01 21:00:00  0.898250  0.898250  0.077167
2013-01-02 00:00:00  0.860000  0.860000  0.075000
                      L06_347  LS06_347  LS06_348
Time                                             
2012-01-01 00:00:00  0.307167  0.273917  0.028000


### Time series resampling

In [107]:
# Average by day
print(data.resample('D').mean().head())
# Sum by year
print(data.resample('Y').sum().head())

             L06_347  LS06_347  LS06_348
Time                                    
2009-01-01  0.125010  0.092281  0.016635
2009-01-02  0.124146  0.095781  0.016406
2009-01-03  0.113562  0.085542  0.016094
2009-01-04  0.140198  0.102708  0.017323
2009-01-05  0.128812  0.104490  0.018167
               L06_347     LS06_347   LS06_348
Time                                          
2009-12-31  640.507333   639.476750  71.634750
2010-12-31  994.468583  1029.896542  75.620100
2011-12-31  704.901583   692.756488  49.946350
2012-12-31  669.086250   659.646667  64.412572
2013-12-31   14.724333    14.724333   1.212583


## 5: String manipulation

### get_dummies()

In [108]:
s = pd.Series(['a','a|b','a|c'])
print(s.str.get_dummies(sep = '|'))

   a  b  c
0  1  0  0
1  1  1  0
2  1  0  1


### split()

In [87]:
s = pd.Series(['a_b_C','c_d_e','f_g_h'])
print(s.str.split('_'))
print(s.str.split('_',expand = True))

0    [a, b, C]
1    [c, d, e]
2    [f, g, h]
dtype: object
   0  1  2
0  a  b  C
1  c  d  e
2  f  g  h


### replace()

In [109]:
df = pd.DataFrame(np.random.randn(3,2),columns = ['A a','B b'],index = range(3))
df.columns = df.columns.str.replace(' ','_')
print(df)

        A_a       B_b
0 -0.960445  0.624197
1  0.791194 -1.353642
2 -0.478425  0.272510
